In [ ]:
!pip install nsetools pandas nsepy PyPortfolioOpt

In [ ]:
from fetcher import Nifty500Fetcher

nff = Nifty500Fetcher()
nff.read_list(update=True)

In [ ]:
nff.fetch_data()

In [ ]:
nff.fetch_metadata()

In [ ]:
from cleaner import Nifty500Cleaner

nfc = Nifty500Cleaner()
nfc.clean_metadata()
nfc.clean_ohlc_data()

In [ ]:
from processor import StockProcessor

sp = StockProcessor('3MINDIA', meta_data_loc='data/cleaned/Metadata/nifty500.json')

In [ ]:
sp.process_metrics()

In [ ]:
sp.stock.metadata

In [ ]:
from processor import IndexProcessor

pp = IndexProcessor("data/cleaned/OHLC/Nifty500", 
                    "data/cleaned/Metadata/nifty500.json", 
                    "data/cleaned/Metadata/proc_nifty500.json")

In [ ]:
pp.process_metrics()

In [ ]:
import pandas as pd
pd.read_csv('data/cleaned/OHLC/Nifty500/M&M.csv')

In [ ]:
from processor import IndexProcessor

pp = IndexProcessor("data/cleaned/OHLC/Nifty500", "data/cleaned/Metadata/nifty500.json")
pp.process_close(30)

In [ ]:
from optimizer import EffOptimizer, HRPOptimizer, CLAOptimizer

opt1 = EffOptimizer(pp)
# opt2 = HRPOptimizer(pp)
# opt3 = CLAOptimizer(pp)

In [ ]:
opt1.optimize_max_sharpe()

In [ ]:
opt1.portfolio.statistics

In [ ]:
opt1.optimize_min_volatility()

In [ ]:
opt1.portfolio.statistics

In [ ]:
opt2 = HRPOptimizer(pp)
opt2.optimize()

In [ ]:
opt2.portfolio.statistics

In [ ]:
temp_df = (pp.close_matrix.pct_change(30).tail(1)*100 > 0)
temp_df

In [ ]:
opt3 = CLAOptimizer(pp)
opt3.optimize_max_sharpe()

In [ ]:
opt3.portfolio.statistics

In [ ]:
opt3.optimize_min_volatility()
opt3.portfolio.statistics

In [ ]:
import pandas as pd
df = pd.read_csv('data/raw/nifty_500_metadata.csv')#.to_dict(orient='records')

In [ ]:
some = df[df['symbol']=='M&amp;M'].to_dict(orient='records')

In [ ]:
some

In [1]:
from fetcher import Nifty500Fetcher

nff = Nifty500Fetcher()
nff.fetch_index('NIFTY 500')

In [1]:
from simulate import Backtesting, Agent
from optimizer import EffOptimizer, HRPOptimizer, CLAOptimizer

agents = [Agent("CLA_MAX_SHARPE", CLAOptimizer(), 1000000, optimizer_type='max_sharpe',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.90),
         Agent("EFF_MIN_VOL", EffOptimizer(), 1000000, optimizer_type='min_vol',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.90),
         Agent("EFF_MAX_SHARPE", EffOptimizer(), 1000000, optimizer_type='max_sharpe',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.90),
         Agent("HRP", HRPOptimizer(), 1000000, metadata_loc="data/processed/Metadata/nifty500.json", 
                           single_day_cash=0.90),
         Agent("CLA_MIN_VOL", CLAOptimizer(), 1000000, optimizer_type='min_vol',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.90)]

benchmarks = [
    {
        'name':'Nifty500',
        'data_loc':'data/raw/Nifty500/raw/Nifty500.csv'
    }
]

bt = Backtesting(proc_ohlc_loc="data/processed/OHLC/Nifty500", 
              proc_metadata_loc="data/processed/Metadata/nifty500.json", bband_margins=True,
                 lookback_period=14,rebalance_period=14,
                 bband_ma=20, #process_metrics=True,
                agents=agents)

In [ ]:
bt.backtest()

In [2]:
!pip install bokeh

In [4]:
import time

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure 
output_notebook()

Loading BokehJS ...

In [5]:
N = 1000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 2
colors = ["#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)]

In [6]:
TOOLS="crosshair,pan,wheel_zoom,box_zoom,reset,tap,box_select,lasso_select"

p = figure(tools=TOOLS)
p.axis.major_label_text_font_size = "24px"
hover = HoverTool(tooltips=None, mode="vline")
p.add_tools(hover)
r = p.circle(x,y, radius=radii, 
             fill_color=colors, fill_alpha=0.6, line_color=None, 
             hover_fill_color="black", hover_fill_alpha=0.7, hover_line_color=None)

In [7]:
target = show(p, notebook_handle=True)

In [ ]:
i = 0
while True:
    i +=1 
    p.title.text = str(i)
    
    r.data_source.data['radius'] = radii * (2 + np.sin(i/5))
    
    x = r.data_source.data['x']
    y = r.data_source.data['y']
    d = np.sqrt((x-50)**2 + (y-50)**2)/100
    rand = 2 * (np.random.random(size=N) - 0.5)
    r.data_source.data['x'] = x + 2 * np.sin(d) * rand
    r.data_source.data['y'] = y + np.cos(d**2) * rand
    
    p.axis.major_label_text_color = r.data_source.data['fill_color'][int(i%N)]

    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
    time.sleep(0.1)